<a href="https://colab.research.google.com/github/tomonari-masada/course2025-nlp/blob/main/04_topic_modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# トピックモデル


* bag-of-wordsの範囲内でテキストデータの高度な分析を行う。
* 教師なしでテキスト集合（＝コーパス）に含まれる多様なトピックを抽出する。
* 今回は、潜在的ディリクレ配分法 (LDA; latent Dirichlet allocation) を使う。
* scikit-learnにある実装を使う。
  * https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.LatentDirichletAllocation.html

* コメント
  * 現在であれば、テキスト埋め込み用のモデル
を使ってベクトルに変換し・・・
  * scikit-learnの適当なクラスタリング手法でクラスタリングする方が、
  * 綺麗に多様なトピックを抽出できるかもしれない。

**以下に示すようなチューニングをしてはじめて、LDAがその能力を発揮してくれます。**

**デフォルトの設定のままでは十分な性能が出ません。**

## 準備

* pyLDAvisというLDAの可視化ツールをインストールする。
  * セッションの再起動が必要かも。

In [ ]:
!pip install pyLDAvis

## データセット
* Hugging Faceにある`CShorten/ML-ArXiv-Papers`を使う。
  * https://huggingface.co/datasets/CShorten/ML-ArXiv-Papers

In [ ]:
from datasets import load_dataset

ds = load_dataset("CShorten/ML-ArXiv-Papers")
ds = ds["train"].train_test_split(test_size=0.1, seed=1234)

In [ ]:
ds

* 今回はタイトルを分析する。

In [ ]:
ds["train"]["title"][:20]

## 単語の出現回数を数える

* LDAを使うにはTF (term frequency) が必要。

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(stop_words="english", min_df=20, max_df=0.5)
X_train = vectorizer.fit_transform(ds["train"]["title"])
X_test = vectorizer.transform(ds["test"]["title"])

In [ ]:
X_train.shape

In [ ]:
X_test.shape

## LDA

* とりあえずLDAの変分推論を動かしてみる。
  * 変分推論 (variational inference) については毎年「統計モデリング２」で説明しています。

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation

lda = LatentDirichletAllocation(
  n_components=20,
  evaluate_every=1,
  verbose=1,
  random_state=123,
)
lda.fit(X_train)

* training setとtest setでperplexityの差が大きい場合、学習がうまくいっていないことが多い。

In [ ]:
lda.perplexity(X_test)

## ハイパーパラメータのチューニング
* perplexityの値が最小になるようにチューニングする。
  * トピック数(`n_components`)は、自分の都合で決めても良いかも。
* トピック数に合わせて、`doc_topic_prior`と`topic_word_prior`の両方をチューニングする。
  * トピック数が変わると、最も良い`doc_topic_prior`と`topic_word_prior`の値も、変わる。

### 1

In [ ]:
for n_components in [20, 30, 40, 50]:
  for doc_topic_prior in [0.2, 0.1, 0.05]:
    for topic_word_prior in [0.05, 0.02, 0.01]:
      lda = LatentDirichletAllocation(
        n_components=n_components,
        doc_topic_prior=doc_topic_prior,
        topic_word_prior=topic_word_prior,
        max_iter=20,
        evaluate_every=1,
        verbose=1,
        random_state=123,
      )
      lda.fit(X_train)
      print(f"-- test perplexity: {lda.perplexity(X_test):.2f}")
      print(f"---- {n_components} topics, alpha={doc_topic_prior:.4f}, eta={topic_word_prior:.4f}")


### 2

In [ ]:
for n_components in [20, 30, 40]:
  for doc_topic_prior in [0.4, 0.3, 0.2]:
    for topic_word_prior in [0.01, 0.005, 0.002]:
      lda = LatentDirichletAllocation(
        n_components=n_components,
        doc_topic_prior=doc_topic_prior,
        topic_word_prior=topic_word_prior,
        max_iter=20,
        evaluate_every=1,
        verbose=1,
        random_state=123,
      )
      lda.fit(X_train)
      print(f"-- test perplexity: {lda.perplexity(X_test):.2f}")
      print(f"---- {n_components} topics, alpha={doc_topic_prior:.4f}, eta={topic_word_prior:.4f}")


### 3

In [ ]:
for n_components in [15, 20, 25]:
  for doc_topic_prior in [0.6, 0.5, 0.4]:
    for topic_word_prior in [0.03, 0.02, 0.01]:
      lda = LatentDirichletAllocation(
        n_components=n_components,
        doc_topic_prior=doc_topic_prior,
        topic_word_prior=topic_word_prior,
        max_iter=20,
        evaluate_every=1,
        verbose=1,
        random_state=123,
      )
      lda.fit(X_train)
      print(f"-- test perplexity: {lda.perplexity(X_test):.2f}")
      print(f"---- {n_components} topics, alpha={doc_topic_prior:.4f}, eta={topic_word_prior:.4f}")


### 4

In [ ]:
for n_components in [10, 15, 20]:
  for doc_topic_prior in [0.8, 0.7, 0.6]:
    for topic_word_prior in [0.03, 0.02, 0.01]:
      lda = LatentDirichletAllocation(
        n_components=n_components,
        doc_topic_prior=doc_topic_prior,
        topic_word_prior=topic_word_prior,
        max_iter=20,
        evaluate_every=1,
        verbose=1,
        random_state=123,
      )
      lda.fit(X_train)
      print(f"-- test perplexity: {lda.perplexity(X_test):.2f}")
      print(f"---- {n_components} topics, alpha={doc_topic_prior:.4f}, eta={topic_word_prior:.4f}")


## 最も良かった設定で改めて変分推論を実行

In [ ]:
vectorizer = CountVectorizer(stop_words="english", min_df=20, max_df=0.5)
X = vectorizer.fit_transform(ds["train"]["title"] + ds["test"]["title"])

In [ ]:
lda = LatentDirichletAllocation(
  n_components=15,
  doc_topic_prior=0.6,
  topic_word_prior=0.02,
  max_iter=50,
  evaluate_every=1,
  verbose=1,
  random_state=123,
)
lda.fit(X)

* モデルを保存

In [ ]:
import pickle

outfile = "lda_model.pk"
with open(outfile, 'wb') as pickle_file:
  pickle.dump(lda, pickle_file)

## 可視化

* LDAの学習時と語彙が同じになるようにする。

In [ ]:
from datasets import load_dataset
from sklearn.feature_extraction.text import CountVectorizer

ds = load_dataset("CShorten/ML-ArXiv-Papers")
ds = ds["train"].train_test_split(test_size=0.1, seed=1234)
vectorizer = CountVectorizer(stop_words="english", min_df=20, max_df=0.5)
vectorizer.fit(ds["train"]["title"])

* データセット全体でterm frequencyを計算しなおす。

In [ ]:
ds = load_dataset("CShorten/ML-ArXiv-Papers")
X = vectorizer.transform(ds["train"]["title"])

* 語彙サイズがLDAの学習時と同じであることを確認する。

In [ ]:
X.shape

In [ ]:
import pickle

outfile = "lda_model.pk"
with open(outfile, "rb") as pickle_file:
  lda = pickle.load(pickle_file)

* pyLDAvisはあらかじめインストールしておく。

In [ ]:
import pyLDAvis
import pyLDAvis.lda_model

pyLDAvis.enable_notebook()
pyLDAvis.lda_model.prepare(lda, X, vectorizer, mds='mmds')